In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import functions as F
from functools import reduce
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
#Python imports
import sys
sys.path.append("/home/svanhmic/workspace/Python/Erhvervs/src/cvr")
import GetNextJsonLayer
import os
path = "/home/svanhmic/workspace/Python/Erhvervs/data/cdata"
#paths


In [2]:
#%ls /home/svanhmic/workspace/Python/Erhvervs/data/cdata 
%env
#%pdb
from graphframes import graphframe

In [3]:
df = sqlContext.read.json(path+"/AlleDeltager.json")
deltagerCols = GetNextJsonLayer.getNextSchemaLayer(df.schema,"deltager")

In [4]:
deltagerCols
df.printSchema()

root
 |-- deltager: struct (nullable = true)
 |    |-- attributter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- sekvensnr: long (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |    |-- vaerdier: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- periode: struct (nullable = true)
 |    |    |    |    |    |    |-- gyldigFra: string (nullable = true)
 |    |    |    |    |    |    |-- gyldigTil: string (nullable = true)
 |    |    |    |    |    |-- sidstOpdateret: string (nullable = true)
 |    |    |    |    |    |-- vaerdi: string (nullable = true)
 |    |    |    |-- vaerditype: string (nullable = true)
 |    |-- beliggenhedsadresse: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- bogstavFra: string (nullable = true)
 |    |    |    |-- bogstavTil: string (nullable = true)
 |    |    |  

In [5]:
deltagerAttributterDf = GetNextJsonLayer.createNextLayerTable(df,["enhedsNummer","enhedstype","navne"],"attributter","deltager")
holdOutCol = ["vaerdier"]
vaerdiCols = [i for i in deltagerAttributterDf.columns if i not in holdOutCol]

deltagerAttributterWithValuesDf = (GetNextJsonLayer
                                   .createNextLayerTable(deltagerAttributterDf,vaerdiCols,holdOutCol[0])
                                   .drop("periode")
                                   .drop("sidstOpdateret")
                                  )
del holdOutCol[0]
holdOutCol.append("navne")
del vaerdiCols
vaerdiCols = [i for i in deltagerAttributterWithValuesDf.columns if i not in holdOutCol]
withNamesAttributsDf = (GetNextJsonLayer
                        .createNextLayerTable(deltagerAttributterWithValuesDf,vaerdiCols,holdOutCol[0])
                        .drop("sidstOpdateret")
                       )
#deltagerAttWithValsAndPeriodDf = GetNextJsonLayer.expandSubCols(df=deltagerAttributterWithValuesDf,mainColumn="periode")
#deltagerAttributterDf.columns


In [6]:
#withNamesAttributsDf.show(truncate=False)

In [7]:
holdtoCols = ["enhedsNummer","enhedstype","navne","stilling"]
delVirkRelDf = (GetNextJsonLayer
                .createNextLayerTable(df,holdtoCols,"virksomhedSummariskRelation","deltager")
                .withColumnRenamed(existing="enhedsNummer",new="personEnhedsNummer")
                .withColumnRenamed(existing="enhedstype",new="personEnhedstype")
                .withColumnRenamed(existing="stilling",new="personStilling")
                .withColumnRenamed(existing="navne",new="personNavne")
                .drop("organisationer")
               )

In [8]:
delVirkRelDf.show(20,truncate=False)
delVirkRelDf.printSchema()

+------------------+----------------+----------------------------------------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
lenUdf = F.udf(lambda x: len(x),IntegerType())

delVirkRelDf.select("personNavne",lenUdf(F.col("personNavne")).alias("length")).orderBy(F.col("length").asc()).show()

In [10]:
delCols = [i for i in delVirkRelDf.columns if i not in ["organisationer","virksomhed"]]
delVirkRelOrgDf =  (GetNextJsonLayer
                    .expandSubCols(delVirkRelDf,"virksomhed")
                    .drop("livsforloeb")
                    .drop(F.col("fejlRegistreret"))
                    .drop("virksomhedsstatus")
                    .drop("virksomhedsform")
                    .drop("sidstIndlaest")
                    .drop("status")
                    .drop("regNummer")
                    .withColumnRenamed(existing="enhedsNummer",new="firmaEnhedsNummer")
                    .withColumnRenamed(existing="navne",new="firmaNavne")
                    .withColumnRenamed(existing="enhedstype",new="firmaEnhedstype")
                    .cache()
                   )

holdOutCol = ["personNavne"]
namesCol = [i for i in delVirkRelOrgDf.columns if i not in holdOutCol]
delVirkRelWithPerNamDf = (GetNextJsonLayer
                          .createNextLayerTable(delVirkRelOrgDf,namesCol,holdOutCol[0])
                          .drop("sidstOpdateret")
                          .withColumnRenamed(existing="navn",new="personNavn")
                         )
del holdOutCol
del namesCol
holdOutCol = ["firmaNavne"]
namesCol = [i for i in delVirkRelWithPerNamDf.columns if i not in holdOutCol]
delVirkRelWithVirkNamDf = (GetNextJsonLayer
                           .createNextLayerTable(delVirkRelWithPerNamDf,namesCol,holdOutCol[0])
                           .drop("sidstOpdateret")
                           .drop("periode")
                           .withColumnRenamed(existing="navn",new="firmaNavn")
                          )

delVirkRelWithVirkNamDf.show()

AnalysisException: 'Cannot resolve column name "firmaNavne" among (personEnhedsNummer, personEnhedstype, personStilling, virksomhed_cvrNummer, virksomhed_enhedsNummer, virksomhed_enhedstype, virksomhed_fejlRegistreret, virksomhed_livsforloeb, virksomhed_navne, virksomhed_regNummer, virksomhed_sidstIndlaest, virksomhed_status, virksomhed_virksomhedsform, virksomhed_virksomhedsstatus, personNavn);'

In [108]:
#create person to oraganisation edge 
personFirmaEdgeDf = (delVirkRelWithVirkNamDf.select(F.col("personEnhedsNummer").alias("src"),F.col("firmaEnhedsNummer").alias("dst")))

In [109]:
personFirmaEdgeDf.show()



+----------+----------+
|       src|       dst|
+----------+----------+
|4000201845|4000873375|
|4004119836|4001457636|
|4004119836|4001457636|
|4000562568|4001580288|
|4000078388|4001021649|
|4000078388|4001021649|
|4000078388|4001021649|
|4000560418|4001565256|
|4000103042|4000856627|
|4000103042|4000856627|
|4000103042|4000856627|
|4000103042|4000856627|
|4003996076|4000945369|
|4003996076|4000842325|
|4000696805|4001835473|
|4004156000|4001560713|
|4000709918|4001863635|
|4004092700|4001370180|
|4004092700|4001370180|
|4000457798|4001393755|
+----------+----------+
only showing top 20 rows



+----------+----------+-------------+
|       src|       dst| relationship|
+----------+----------+-------------+
|4000201845|         0|Interessenter|
|4004119836|4004954366|   Bestyrelse|
|4000562568|         0|Interessenter|
|4000078388|         0|Interessenter|
|4000560418|         0|Interessenter|
|4000103042|         0|Interessenter|
|4003996076|4004503102|    Direktion|
|4003996076|4004411703|    Direktion|
|4000696805|         0|Interessenter|
|4004156000|4005087331|     Stiftere|
|4004156000|4005087332|   Bestyrelse|
|4000709918|         0|Interessenter|
|4004092700|4004857561|   Bestyrelse|
|4000457798|         0|Interessenter|
|4000457798|         0|Interessenter|
|4000408635|         0|Interessenter|
|4000611422|         0|Interessenter|
|4000269638|         0|Interessenter|
|4004131896|4005243831|     Stiftere|
|4004131896|4005243832|    Direktion|
+----------+----------+-------------+
only showing top 20 rows



In [60]:
allPersons = (virkSumRelationWithNamesDf
                  .select(F.col("enhedsNummer").alias("id")
                          ,F.col("navne_navn").alias("navn")
                          ,F.lit("Person").alias("type"))
                  .distinct()
                  .cache()
                 )
allOrgs = (virkSumRelationWithNamesDf
           .select(F.col("enhedsNummerOrganisation").alias("id")
                   ,F.col("orgNavn_navn").alias("navn")
                   ,F.lit("Organisation").alias("type"))
           .filter(F.col("id") !=0)
           .cache()
          )
allVerticesDf = allOrgs.unionAll(allPersons)

In [61]:
allPersons.count()
allOrgs.count()

2820218

In [56]:
personOrgGrsaphDf = graphframe.GraphFrame(allVerticesDf,personTilFirmaEdgeDf).cache()
groupedCols = [F.col("type")]
personOrgGraphDf.vertices.groupby(*groupedCols).count().show()

+------------+-------+
|        type|  count|
+------------+-------+
|      Person|3773220|
|Organisation|3773220|
+------------+-------+



In [27]:
personOrgGraphDf.degrees.show()

+----------+------+
|        id|degree|
+----------+------+
|4004163266|     3|
|4004383649|    14|
|4005812438|     4|
|4004826402|     4|
|4004831974|     3|
|4004203683|     1|
|4004821174|    13|
|4004712825|    28|
|4000219055|     2|
|4005813839|     2|
|4005782118|     3|
|4004274696|     1|
|4004746497|     1|
|4005809630|     6|
|4005806416|     1|
|4004702564|    10|
|4005810208|     1|
|4005811904|     2|
|4005813403|     4|
|4004382609|     9|
+----------+------+
only showing top 20 rows



In [30]:
personOrgGraphDf.outDegrees

+----------+---------+
|        id|outDegree|
+----------+---------+
|4004163266|        3|
|4004203683|        1|
|4000219055|        2|
|4004274696|        1|
|4000238588|        2|
|4004072379|        1|
|4000349745|        2|
|4004146496|        1|
|4000052652|        2|
|4004248640|        1|
|4004212823|        2|
|4000312242|        4|
|4004262639|        1|
|4004205560|        1|
|4000338938|        1|
|4000305565|        1|
|4003946989|        1|
|4000394288|        3|
|4000394312|        1|
|4000000978|        2|
+----------+---------+
only showing top 20 rows



In [34]:
personOrgGraphDegreesDf = personOrgGraphDf.vertices.join(personOrgGraphDf.outDegrees,(personOrgGraphDf.vertices["id"]==personOrgGraphDf.outDegrees["id"]),"inner")

In [35]:
personOrgGraphDegreesDf.show()

+-------+--------------------+------+-------+---------+
|     id|                navn|  type|     id|outDegree|
+-------+--------------------+------+-------+---------+
|1859433| Lars Stampe Nielsen|Person|1859433|        3|
|1859433| Lars Stampe Nielsen|Person|1859433|        3|
|1859433| Lars Stampe Nielsen|Person|1859433|        3|
|2164870|Jørgen Herman Jan...|Person|2164870|        2|
|2164870|Jørgen Herman Jan...|Person|2164870|        2|
|2249965|    Jakob Lee Jensen|Person|2249965|       19|
|2249965|    Jakob Lee Jensen|Person|2249965|       19|
|2249965|    Jakob Lee Jensen|Person|2249965|       19|
|2249965|    Jakob Lee Jensen|Person|2249965|       19|
|2249965|    Jakob Lee Jensen|Person|2249965|       19|
|2249965|    Jakob Lee Jensen|Person|2249965|       19|
|2249965|    Jakob Lee Jensen|Person|2249965|       19|
|2249965|    Jakob Lee Jensen|Person|2249965|       19|
|2249965|    Jakob Lee Jensen|Person|2249965|       19|
|2249965|    Jakob Lee Jensen|Person|2249965|   